## Importing Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display


### Cleaning and formatting data

In [2]:
#Importing the data
filename = "https://github.com/Nayrbnat/EC1B1-Coursework/raw/main/International_Financial_Statistics.xlsx"
df = pd.read_excel(filename, na_filter = False,header=0)

#Cleaning the data
df = df.transpose()
df.columns = df.iloc[1]
df = df.rename_axis('Date')
df = df[df.index.str.match(r'^[A-Z][a-z]{2} \d{4}$')]
df = df.iloc[0:, 0:].apply(pd.to_numeric) #Changing the data to numeric datatype

#Separating germany and USA data from the dataframes
df_germany = df.iloc[:,0:4]
df_usa = df.iloc[:,4:8]

#Changing the index to date time format
df_germany.index = pd.to_datetime(df_germany.index, format='%b %Y')
df_usa.index = pd.to_datetime(df_germany.index, format='%b %Y')

### Constructing Germany Dataframe

In [3]:
#Inserting new column for monthly growth in nominal exchange rate

df_germany.insert(loc=2, column='Nominal_Monthly_Growth_Exchange_Rate', value = df_germany['Exchange Rates, US Dollar per Domestic Currency, Period Average, Rate'].pct_change() * 100)

#Inserting new column for real exchange rate
#Real Exchange rate = Nominal Exchange Rate in marks/dollars * germany price level / US price level
germany_real_exchange_rate = df_germany['Exchange Rates, US Dollar per Domestic Currency, Period Average, Rate'] * df_germany['Prices, Consumer Price Index, All items, Index'] /df_usa['Prices, Consumer Price Index, All items, Index']
df_germany.insert(loc=5, column='Real_Monthly_Exchange_Rate', value = germany_real_exchange_rate)

#Inserting new column for monthly growth in real exchange rate
df_germany.insert(loc=6, column='Real_Monthly_Growth_Exchange_Rate', value =df_germany['Real_Monthly_Exchange_Rate'].pct_change() * 100)

#Inserting new column for The monthly growth in industrial production
df_germany.insert(loc=1, column='Monthly_Growth_Industrial_Production', value = df_germany['Economic Activity, Industrial Production, Index'].pct_change() * 100)

#Inserting new column for The growth in industrial production versus 12 months ago
df_germany.insert(loc=1, column='Yearly_Growth_Industrial_Production', value = df_germany['Economic Activity, Industrial Production, Index'].pct_change(12) * 100)

#Setting an index of the value of national reserves
df_germany.insert(loc=7, column='Index_National_Reserves', value = (df_germany['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] / df_germany['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].iloc[0] * 100).fillna(100))

#wewe #Inserting a new column for the monthly inflation rate
df_germany.insert(loc=6, column ='Monthly_Inflation_Rate', value = df_germany['Prices, Consumer Price Index, All items, Index'].pct_change() * 100)

### Real Exchange Rate Calculation

$$ RER_{Germany} = NER_{Germany} * \frac{CPI_{Germany}}{CPI_{USA}} $$

where RER = Real Exchange Rate; NER = Nominal Exchange Rate

### Indexing National Reserves

$$ Base Month: Jan 1960 = 100 $$
  

$$  National\,Reserves\,index_{n} = \frac{National\,Reserve\,Value_{n}}{National\,Reserve\,Value_{Jan 1960}} * 100 $$

### Constructing USA Dataframe

In [4]:
#Inserting new column for the monthly inflation rate
df_usa.insert(loc=4, column='Monthly_Inflation_Rate', value = df_usa["Prices, Consumer Price Index, All items, Index"].pct_change() * 100)

#Setting an index of the value of national reserves
df_usa.insert(loc=3, column='Index_National_Reserves', value = (df_usa['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'] / df_usa['International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar'].iloc[0] * 100).fillna(100))

In [5]:
#Lets view the germany data frame
df_germany


Unnamed: 1,"Economic Activity, Industrial Production, Index",Yearly_Growth_Industrial_Production,Monthly_Growth_Industrial_Production,"Exchange Rates, US Dollar per Domestic Currency, Period Average, Rate",Nominal_Monthly_Growth_Exchange_Rate,"International Reserves and Liquidity, Reserves, Official Reserve Assets, US Dollar",Monthly_Inflation_Rate,"Prices, Consumer Price Index, All items, Index",Index_National_Reserves,Real_Monthly_Exchange_Rate,Real_Monthly_Growth_Exchange_Rate
Date,,,,,,,,,,,
1960-01-01,31.214706,NaN,NaN,0.238095,NaN,4724.155785,NaN,24.616929,100.000000,0.436198,NaN
1960-02-01,31.062322,NaN,-0.488181,0.238095,0.000000,4806.362830,-0.568147,24.477068,101.740143,0.432245,-0.906350
1960-03-01,32.225254,NaN,3.743868,0.238095,0.000000,4966.456016,0.000000,24.477068,105.128964,0.432245,0.000000
1960-04-01,34.310512,NaN,6.470881,0.238095,0.000000,5236.120624,0.571393,24.616929,110.837171,0.433241,0.230473
1960-05-01,33.749096,NaN,-1.636279,0.238095,0.000000,5417.091847,0.284096,24.686864,114.667934,0.434472,0.284096
...,...,...,...,...,...,...,...,...,...,...,...
1990-08-01,76.015669,6.017897,-3.590683,0.636659,4.405679,72425.738573,0.311532,67.556702,1533.093782,0.712667,3.775943
1990-09-01,86.554241,5.514275,13.863684,0.637064,0.063706,73197.573621,0.310565,67.766509,1549.431834,0.709406,-0.457573
1990-10-01,92.705752,6.152998,7.107116,0.656470,3.046019,75011.926830,0.722391,68.256049,1587.837706,0.731883,3.168449


In [ ]:
#Lets view the usa data frame
df_usavv  


## Identifying Outliers

In [ ]:
#The code below attempts to remove the outliers and set the values as NA
def get_outliers_lists(df):
    outliers_df = pd.DataFrame()
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outlier_values = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column].tolist()
        
        if outlier_values:  # Check if there are any outliers
            outlier_values_df = pd.DataFrame({'Outlier_Value': outlier_values, 'Outlier_Column': column})
            outlier_values_df.index = df.index[df[column].isin(outlier_values)]
            outliers_df = pd.concat([outliers_df, outlier_values_df])
    
    return outliers_df

**Explaining the function**

$$IQR = Interquartile\,range$$

Our criteria for an outlier is adopted from the Tukey's fences method. This refers to data points that lie 1.5 * IQR below the first quartile (Q1) or above the third quartile (Q3).

This function goes through data from columns which are of the numerical datatype and classifies them based on the criteria above and adds them to a list named outlier_values.

The IF function then ascertains if there are any outliers and subsequently adds them to a dataframe with the index being the data

In [ ]:
def replace_outliers_with_na(df, outliers_df):
    modified_df = df.copy()  # Create a copy of the original DataFrame to modify
    
    for index, row in outliers_df.iterrows():
        modified_df.loc[index, row['Outlier_Column']] = np.nan
    return modified_df

**Explaining the function**

This function takes 2 arguments: the dataframe we want to augment and the list of outliers. We structured our code in this format because it allows us to change the outlier dataframe if needed. Some of the outliers might not be "true" outliers, hence some discretion is available.

This function iterates through every row and sets any outliers as NA

In [ ]:
#Creating a df of outliers for germany
germany_outliers = get_outliers_lists(df_germany)
germany_outliers

In [ ]:
#Creating a df of outliers for USA
usa_outliers = get_outliers_lists(df_usa)
usa_outliers

Although there might be outliers in the data. It would be asinine to simply set these values as NA without understanding the circumstances behind them. Hence, we decided to extract the values into a dataframe and analyse them. Based on this we realised that the period of outliers was from XX to YY
We conclude that this was due to ZZ.
Hence we decided to remove the data as this was a result of {insert supply shock,etc here}

In [ ]:
#Creating a separate dataframe setting extreme values as NA for germany
df_modified_germany = replace_outliers_with_na(df_germany,germany_outliers)
df_modified_germany

In [ ]:
#Creating a separate dataframe setting extreme values as NA for USA
df_modified_usa = replace_outliers_with_na(df_usa, usa_outliers)
df_modified_usa

## Interpolating Missing Data

In [ ]:
#Creating function to interpolate data

def interpolate_missing_data(df):
    interpolated_df = df.copy()
    
    for column in interpolated_df.select_dtypes(include=[np.number]).columns:
        if interpolated_df[column].isnull().any():
            interpolated_df[column] = interpolated_df[column].interpolate(method='linear', limit_direction='both')
    
    return interpolated_df

**Explaining Interpolating Function**

This function basically creates a copy of the dataframe we input and then iterates through each column (which are numerical). It then interpolates the data with the interpolate() function, using specific arguments given in order to calculate the average values based on the next and previous data points.

In [ ]:
#Interpolating missing data for germany
interpolate_missing_data(df_modified_germany)

In [ ]:
#Interpolating missing data for germany
interpolate_missing_data(df_modified_usa)

# Analysis